In [21]:
# import pandas as pd
# import numpy as np
# from sklearn.ensemble import IsolationForest
# from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, confusion_matrix
# from sklearn.model_selection import GridSearchCV
# import matplotlib.pyplot as plt
# import json
# from datetime import datetime

# # Data laden
# data = pd.read_json('./generated_visits.json')

# # Timestamp voor wanneer het model draait
# run_at = datetime.now().isoformat()

# # Extract relevant features for anomaly detection
# data['visitDate'] = pd.to_datetime(data['visit_timestamp'])
# data['visit_month'] = data['visitDate'].dt.month
# data['visit_year'] = data['visitDate'].dt.year
# data['visit_day'] = data['visitDate'].dt.day
# data['visit_hour'] = data['visitDate'].dt.hour

# # Extract visit related data
# data['visitId'] = data['visit'].apply(lambda x: x['id'])
# data['visitAmounts'] = data['visit'].apply(lambda x: x['visitAmounts'])
# data['duration'] = data['visit'].apply(lambda x: x['duration'])
# data['service'] = data['visit'].apply(lambda x: x['service'])
# data['nurse_latitude'] = data['visit'].apply(lambda x: x['nurseLocation']['latitude'])
# data['nurse_longitude'] = data['visit'].apply(lambda x: x['nurseLocation']['longtitude'])
# data['patient_latitude'] = data['visit'].apply(lambda x: x['patientLocation']['latitude'])
# data['patient_longitude'] = data['visit'].apply(lambda x: x['patientLocation']['longtitude'])

# # Convert categorical features to numeric
# data = pd.get_dummies(data, columns=['duration', 'service'])

# # Select relevant features
# features = ['visitAmounts', 'visit_month', 'visit_year', 'visit_day', 'visit_hour',
#             'nurse_latitude', 'nurse_longitude', 'patient_latitude', 'patient_longitude']
# features += [col for col in data.columns if col.startswith('duration_')]
# features += [col for col in data.columns if col.startswith('service_')]

# X = data[features].values

# # Hyperparameter tuning for Isolation Forest using GridSearchCV
# param_grid = {
#     'n_estimators': [50, 100, 150],
#     'max_samples': ['auto', 0.9, 0.75],
#     'contamination': [0.05, 0.1, 0.15],
#     'max_features': [1.0, 0.9, 0.75]
# }

# iso_forest = IsolationForest(random_state=42)
# grid_search = GridSearchCV(iso_forest, param_grid, cv=5, scoring='f1')
# grid_search.fit(X)

# # Beste hyperparameters
# best_params = grid_search.best_params_
# print("Beste hyperparameters:", best_params)

# # Train het model met de beste hyperparameters
# best_iso_forest = IsolationForest(**best_params, random_state=42)
# best_iso_forest.fit(X)

# # Voorspel anomalieën
# data['anomaly'] = best_iso_forest.predict(X)
# data['anomaly'] = data['anomaly'].apply(lambda x: 1 if x == -1 else 0)

# # Tel anomalieën
# anomaly_count = data['anomaly'].sum()
# print(f"Aantal anomalieën gedetecteerd: {anomaly_count}")

# # Simulatie van 'true labels' voor demonstratiedoeleinden (deze moet je vervangen door echte labels indien beschikbaar)
# # We nemen aan dat alle visitAmounts boven het 75e percentiel anomalieën zijn
# threshold = data['visitAmounts'].quantile(0.75)
# data['true_label'] = (data['visitAmounts'] > threshold).astype(int)

# # Prestatiemetingen
# accuracy = accuracy_score(data['true_label'], data['anomaly'])
# recall = recall_score(data['true_label'], data['anomaly'])
# precision = precision_score(data['true_label'], data['anomaly'])
# f1 = f1_score(data['true_label'], data['anomaly'])

# print(f'Accuracy: {accuracy:.2f}')
# print(f'Recall: {recall:.2f}')
# print(f'Precision: {precision:.2f}')
# print(f'F1 Score: {f1:.2f}')

# # Confusion matrix
# cm = confusion_matrix(data['true_label'], data['anomaly'])
# fig, ax = plt.subplots()
# cax = ax.matshow(cm, cmap=plt.cm.Blues)
# fig.colorbar(cax)
# ax.set_xlabel('Predicted')
# ax.set_ylabel('True')
# ax.set_xticklabels(['', 'Normal', 'Anomaly'])
# ax.set_yticklabels(['', 'Normal', 'Anomaly'])
# plt.show()

# # Visualiseer de resultaten
# plt.figure(figsize=(10, 6))
# plt.scatter(data['visitDate'], data['visitAmounts'], c=data['anomaly'], cmap='coolwarm', alpha=0.6)
# plt.xlabel('Visit Date')
# plt.ylabel('Visit Amounts')
# plt.title('Detected Anomalies in Visit Amounts')
# plt.show()

# # Aanvullende checks op basis van de extra cases
# anomalies = []

# # Controleer op zorg door meerdere verpleegkundigen op dezelfde dag
# for patient_id, group in data.groupby('rijksregisterPatient'):
#     if group['visitDate'].dt.date.nunique() > 1:
#         anomalies.append(f"Meerdere verpleegkundigen zorgden voor patiënt {patient_id} op dezelfde dag")

# # Controleer op verdachte uren
# for i, row in data.iterrows():
#     hour = row['visitDate'].hour
#     if hour < 6 or hour > 22:
#         anomalies.append(f"Verdachte uur voor bezoek aan patiënt {row['rijksregisterPatient']} door verpleegkundige {row['rijksregisterNurse']}")

# # Controleer op te korte tussenpozen tussen bezoeken
# for patient_id, group in data.groupby('rijksregisterPatient'):
#     for i in range(1, len(times)):
#         if (times.iloc[i] - times.iloc[i-1]).total_seconds() < 2.5 * 3600:
#             anomalies.append(f"Te korte tussenpozen tussen bezoeken aan patiënt {patient_id}")

# # Maak de gewenste outputstructuur
# output = []
# for index, row in data.iterrows():
#     if row['anomaly'] == 1:
#         output.append({
#             "type": "VISIT_CHECK",
#             "rijksregisterNurse": row["rijksregisterNurse"],
#             "rijksregisterPatient": row["rijksregisterPatient"],
#             "run_at": run_at,
#             "weight": row["visitAmounts"],
#             "visitAmounts": row['visitAmounts'],
#             "visit date": row["visitDate"]
#             "visitId": row["visitId"]
#         })

# # Voeg anomalieën toe aan de output
# for anomaly in anomalies:
#     output.append({
#         "type": "VISIT_CHECK",
#         "rijksregisterNurse": "",
#         "rijksregisterPatient": "",
#         "weight": 0,
#         "visitId": 0,
#         "anomaly_description": anomaly
#     })

# # Exporteer de output naar een JSON-object
# output_json = json.dumps(output, indent=4)
# print("JSON Output van gedetecteerde anomalieën:")
# print(output_json)


import pandas as pd

# Gegevens inladen
data = pd.read_json('./generated_visits.json')

# Tijdsstempels converteren naar datetime
data['visit_timestamp'] = pd.to_datetime_ext(data['visit_timestamp'])

# Functie om verdachte scanuren te controleren
def check_strange_hours(data):
    return data[data['visit_timestamp'].dt.hour > 22]

# Functie om frequente zorg op dezelfde dag te detecteren
def check_frequent_visits(data):
    daily_visits = data.groupby(['rijksregisterPatient', data['visit_timestamp'].dt.date]).size()
    return daily_visits[daily_visits > 3]

# Functie om te controleren of dezelfde zorg door verschillende zorgverleners op dezelfde dag wordt verleend
def check_multiple_nurses(data):
    service_per_day = data.groupby(['rijksregisterPatient', 'service', data['visit_timestamp'].dt.date])['rijksregisterNurse'].nunique()
    return service_per_day[service_per_day > 1]

# Afstandsberekening tussen twee locaties
from geopy.distance import geodesic

# Functie om de afstand te controleren
def check_location_mismatch(data):
    suspicious_distance = []
    for index, row in data.iterrows():
        nurse_loc = (row['visit']['nurseLocation']['latitude'], row['visit']['nurseLocation']['longtitude'])
        patient_loc = (row['visit']['patientLocation']['latitude'], row['visit']['patientLocation']['longtitude'])
        if geodesic(nurse_loc, patient_loc).kilometers > 50:  # Meer dan 50 km afstand is verdacht
            suspicious_distance.append(row['visit']['id'])
    return suspicious_distance

# Pas deze functies toe om de respectievelijke checks uit te voeren
strange_hours = check_strange_hours(data)
frequent_visits = check_frequent_visits(data)
multiple_nurses = check_multiple_nurses(data)
location_mismatches = check_location_mismatch(data)

# Resultaten printen
print("Strange Hours Visits:", strange_hours)
print("Frequent Visits:", frequent_visits)
print("Multiple Nurses Same Day Service:", multiple_nurses)
print("Location Mismatches:", location_mismatches)



AttributeError: module 'pandas' has no attribute 'to_datetime_ext'